In [424]:
#Librerias
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from google.cloud import bigquery
from pandas_gbq import read_gbq
from sklearn.linear_model import LinearRegression
import joblib
import warnings 
warnings.filterwarnings("ignore")
import pickle


In [425]:
#Carga de datos desde la nube
project_id = 'mystic-cable-435413-b4'
query = """
SELECT * 
FROM `mystic-cable-435413-b4.googleyelp.yelp-business`
"""

#Cargar los datos desde BigQuery usando pandas-gbq
df = pd.read_gbq(query, project_id=project_id)


In [427]:
# 2. Análisis de Sentimiento (suponiendo que ya tienes las columnas de sentimiento)
df['avg_sentiment'] = df[['reviews_sentiment', 'tips_sentiment']].mean(axis=1).fillna(0)

# 3. Agrupación por cluster en latitud y longitud
X_geo = df[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X_geo)

# 4. Combinar puntaje por stars y analisis de sentimiento (multiplicado por 7.5 para igualar importancia con stars)
df['combined_score'] = df['stars'] + df['avg_sentiment'] * 7.5


In [428]:
# Obtener el valor mínimo y máximo del combined_score
min_combined_score = df['combined_score'].min()
max_combined_score = df['combined_score'].max()

# Normalizar el combined_score entre 1 y 10
df['combined_score'] = 1 + (df['combined_score'] - min_combined_score) * (10 - 1) / (max_combined_score - min_combined_score)

In [429]:
# Agrupar negocios por 'main category', 'categories' y 'cluster', cálculo de puntaje promedio
df_grouped = df.groupby(['main category', 'categories', 'cluster']).agg({
    'combined_score': 'mean',   
    'stars': 'mean',            
    'avg_sentiment': 'mean',    
    'business_id': 'count',     
    'latitude': 'mean',         
    'longitude': 'mean',        
    'postal_code': 'first',
    
}).reset_index()


# 5. Machine Learning Training/Testing
# Variables usadas:
X = df[['latitude', 'longitude', 'stars', 'avg_sentiment', 'postal_code']]
y = df['combined_score']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado de datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 9. Modelo XGBoost para ajuste de hiperparametros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

model = XGBRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# 10. Evaluacion del modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)




# 11. Contribucion de variables
feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importancia']).sort_values('importancia', ascending=False)
print("Evaluacion de importancia:")
print(feature_importances)

Evaluacion de importancia:
               importancia
stars             0.536435
avg_sentiment     0.462141
postal_code       0.000678
longitude         0.000378
latitude          0.000368


In [430]:
# Guardar el modelo de XGBoost
best_model.save_model("xgboost_model.json")

# Guardar el scaler con pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [431]:
# 13. Encontrar las 5 top categorías predichas por el modelo entrenado

# Utilizar las variables de entrada que has definido antes como 'X'
X_for_prediction = df_grouped[['latitude', 'longitude', 'stars', 'avg_sentiment', 'postal_code']]

# Escalar los datos como en el entrenamiento
X_for_prediction_scaled = scaler.transform(X_for_prediction)

# Predecir el combined_score usando el modelo entrenado
df_grouped['predicted_combined_score'] = best_model.predict(X_for_prediction_scaled)

# Seleccionar las 5 categorías con el mejor combined_score predicho
top_5_categories = df_grouped.groupby('main category').agg({
    'predicted_combined_score': 'mean'
}).reset_index().nlargest(5, 'predicted_combined_score')

# Resetear el índice para una mejor visualización
top_5_categories = top_5_categories.reset_index(drop=True)

# Imprimir las 5 top categorías en orden de puntaje
print("Top 5 Categorias - Puntaje (Predicción del modelo):")
for i, row in top_5_categories.iterrows():
    print(f"{i+1}. {row['main category']} -  {row['predicted_combined_score']:.2f}")


Top 5 Categorias - Puntaje (Predicción del modelo):
1. Beauty & Spa -  7.17
2. Active life -  6.96
3. Pets -  6.88
4. Cofee Shop -  6.86
5. Shopping -  6.77


In [458]:
# 14. Seleccionar una de las top 5 categorías
selected_main_category = input("Por favor, selecciona una de las top 5 main categories: ")

# 15. Encontrar las mejores 2 subcategorías dentro de la main category seleccionada
df_filtered = df_grouped[df_grouped['main category'] == selected_main_category]

# Agrupar por 'categories' dentro de la 'main category' seleccionada utilizando el combined score predicho
top_2_subcategories = df_filtered.groupby('categories').agg({
    'predicted_combined_score': 'mean'  # Usar el combined_score predicho por el modelo
}).reset_index().nlargest(2, 'predicted_combined_score')

# Imprimir las 2 mejores subcategorías
print(f"Top 2 Subcategorías para '{selected_main_category}':")
for i, row in enumerate(top_2_subcategories.itertuples(), 1):
    print(f"{i}. {row.categories} - {row.predicted_combined_score:.2f}")


Top 2 Subcategorías para 'Shopping':
1. Tennis - 9.03
2. Spray Tanning - 8.69


In [460]:
# Pedir al usuario que seleccione una subcategoría
selected_subcategory_index = int(input("\nPor favor selecciona una subcategoría (1 o 2): ")) - 1
selected_subcategory = top_2_subcategories.iloc[selected_subcategory_index]['categories']

# 18. Distancia Haversine para evitar que las ubicaciones sean iguales
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    d_lat = np.radians(lat2 - lat1)
    d_lon = np.radians(lon2 - lon1)
    a = np.sin(d_lat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(d_lon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c * 1000  # Convertir a metros

# 19. Predecir la primera ubicación basada en la subcategoría seleccionada usando el modelo entrenado
def recommend_top_1_location_for_subcategory(subcategory):
    subcategory_data = df_grouped[df_grouped['categories'] == subcategory]
    
    # Seleccionar las características necesarias para la predicción
    X_subcategory = subcategory_data[['latitude', 'longitude', 'stars', 'avg_sentiment', 'postal_code']]
    
    # Escalar las características de la misma manera que durante el entrenamiento
    X_subcategory_scaled = scaler.transform(X_subcategory)
    
    # Predecir el combined_score usando el modelo entrenado
    subcategory_data['predicted_combined_score'] = best_model.predict(X_subcategory_scaled)
    
    # Seleccionar la mejor ubicación basada en la predicción del modelo
    top_1_location = subcategory_data.nlargest(1, 'predicted_combined_score')
    return top_1_location

# 20. Predecir la segunda ubicación basada en análisis de subcategorías similares usando el modelo entrenado y Haversine
def recommend_top_1_location_from_similar_categories(subcategory, top_1_lat, top_1_lon):
    # Filtrar categorías con análisis de sentimiento similar
    similar_categories_data = df_grouped[
        (df_grouped['categories'] != subcategory) & 
        (abs(df_grouped['avg_sentiment'] - df_grouped[df_grouped['categories'] == subcategory]['avg_sentiment'].mean()) < 0.1)
    ]
    
    # Si se encuentran categorías similares
    if len(similar_categories_data) > 0:        # Seleccionar las características necesarias para la predicción
        X_similar_categories = similar_categories_data[['latitude', 'longitude', 'stars', 'avg_sentiment', 'postal_code']]
        
        # Escalar las características
        X_similar_categories_scaled = scaler.transform(X_similar_categories)
        
        # Predecir el combined_score usando el modelo entrenado
        similar_categories_data['predicted_combined_score'] = best_model.predict(X_similar_categories_scaled)
        
        # Calcular la distancia desde la primera ubicación
        similar_categories_data['distance'] = similar_categories_data.apply(
            lambda row: haversine_distance(top_1_lat, top_1_lon, row['latitude'], row['longitude']), axis=1
        )
        
        # Seleccionar la mejor ubicación de categorías similares que esté a más de 1000 metros (1 km)
        top_1_other_location = similar_categories_data[similar_categories_data['distance'] > 1000].nlargest(1, 'predicted_combined_score')
        
        return top_1_other_location
    else:
        # Si no se encuentran categorías similares, seleccionamos por puntaje más cercano
        return df_grouped[df_grouped['categories'] != subcategory].nlargest(1, 'predicted_combined_score')

# 21. Obtener la primera ubicación basada en la subcategoría seleccionada
top_1_location_subcategory = recommend_top_1_location_for_subcategory(selected_subcategory)
top_1_location_subcategory = top_1_location_subcategory.reset_index(drop=True)

# Extraer latitud y longitud de la primera ubicación
top_1_lat = top_1_location_subcategory['latitude'].iloc[0]
top_1_lon = top_1_location_subcategory['longitude'].iloc[0]

# 22. Obtener la segunda ubicación basada en categorías similares y la distancia Haversine
top_1_location_similar_category = recommend_top_1_location_from_similar_categories(selected_subcategory, top_1_lat, top_1_lon)
top_1_location_similar_category = top_1_location_similar_category.reset_index(drop=True)

# 23. Mostrar ambas ubicaciones con índices comenzando en 1
top_2_locations = pd.concat([top_1_location_subcategory, top_1_location_similar_category], ignore_index=True)

# Asignar índices comenzando en 1
top_2_locations.index = top_2_locations.index + 1

# Mostrar las ubicaciones con índices 1 y 2
print(f"\nPosibles Ubicaciones para la subcategoría '{selected_subcategory}':")
print(top_2_locations[['postal_code']])



Posibles Ubicaciones para la subcategoría 'Spray Tanning':
   postal_code
1        19107
2        19122


## Error Cuadratico Medio (MSE)

____________________

## ANALISIS DE METRICAS

In [404]:
# Calculo Error Cuadratico Medio en Test (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.0016384779306283593


### OBS: Un bajo valor de MSE implica que las predicciones son cercanas al valor real, buena precision y los datos en la muestra son consistentes puesto que este parametro es muy suceptible a outliers.

## R^2

In [405]:
from sklearn.metrics import r2_score
y_train_pred = best_model.predict(X_train_scaled)
train_r2 = r2_score(y_train, y_train_pred)
print(f"Train R-squared: {train_r2}")

r2 = r2_score(y_test, y_pred)
print(f"Test R-squared: {r2}")

Train R-squared: 0.9998715804529332
Test R-squared: 0.9988899789692228


### OBS: Se pensaria que este modelo en particular esta sobre entrenado ya que el Train R^2 es cercano a 1, sin embargo el Test R^2 es igualmente alto cercano a 1. Al ambos ser similares y proximos a 1, indica que el modelo esta generalizando bien con los datos recibidos desde Google Cloud Platform (GCP)

## Evaluacion por: Validacion Cruzada

In [406]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# Parametros para la validacion
xgb_model = XGBRegressor(
    n_estimators=50,     # Numero de arboles
    max_depth=3,         # profundidad de arboles
    learning_rate=0.1,   # Tasa de aprendizaje moderado
    random_state=42,     # Aleatoriedad
    n_jobs=-1            # Usar todos los nucleos para el analisis
)

# Desempeno de la validacion K-fold= 3 (cv=3 para optimizar el uso de memoria y tiempo de analisis)
cv_scores = cross_val_score(xgb_model, X_train_scaled, y_train, cv=3, scoring='r2')

# Cálculo de la desviación estándar
std_dev = np.std(cv_scores)

print(f"Puntaje Cross-validation R²: {cv_scores}")
print(f"Media de la Cross-validation R²: {np.mean(cv_scores)}")
print(f"Desviación estándar de la Cross-validation R²: {std_dev}")

Puntaje Cross-validation R²: [0.99651997 0.99652959 0.99721681]
Media de la Cross-validation R²: 0.9967554578151828
Desviación estándar de la Cross-validation R²: 0.00032624815404414003


### OBS: Metodo usado para evaluar la variabilidad y la confiabilidad de modelos de ML. Entre mas altos y cercanos sean los valores de validacion, mejor el desempeño. Ya que los 3 valores de la validacion son consistentes y similares a lo largo de la muestra, indica que es capaz de generalizar sin estar sobre entrenado y con una baja desviacion estandar.